# STOCK MARKET SHORT PREDICTION MODEL

@author: Matheus José Oliveira dos Santos

## 1. Imports

In [33]:
import sys
import os
from datetime import date, datetime

import pandas as pd
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

sys.path.append(os.getcwd()+'\\..\\src')
import db_interface

## 2. Get data

In [3]:
my_db = db_interface.DB_interface("FINANCE_DB")

with my_db:
    table_name = "financials"
    query = "SELECT * FROM {0} WHERE date BETWEEN '{1}' AND '{2}'".format(table_name, '2003-01-01', '2023-08-10')
    df_raw = my_db.read_by_command(query)

connecting in: FINANCE_DB


C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\src\db_interface.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(command, self.db)


DB Closed


## 3. Pre Processing

In [4]:
# Describe raw data
df_raw.describe()

,PX_LAST,EQY_SH_OUT,CURR_ENTP_VAL,market_cap,net_income,EBITDA,EBIT,BS_TOT_ASSET,BS_TOT_LIAB2,equity,...,EV/EBITDA_std_1y,EV/EBITDA_std_3y,EV/EBITDA_std_5y,EV/EBIT_std_6m,EV/EBIT_std_1y,EV/EBIT_std_3y,EV/EBIT_std_5y,return_3m,return_6m,return_1y
count,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,3.082900e+05,308290.000000,308290.000000,...,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000,308290.000000
mean,36.287864,1146.681379,29080.755882,18039.165750,338.681589,914.024794,677.792130,2.726070e+04,16293.794918,10966.908232,...,2.869103,4.124008,4.135563,10.686458,15.579769,22.024584,21.527782,0.023009,0.052037,0.126177
std,214.761366,2005.126975,72559.526422,42963.301242,2558.577792,4575.272361,3692.827056,9.250951e+04,57263.195415,37516.705739,...,10.840442,11.916046,11.810097,151.157049,196.658228,206.126792,169.693442,0.263221,0.360445,0.586478
min,0.000000,0.000000,-1007.310000,0.000000,-48523.000000,-28243.000000,-44001.000000,0.000000e+00,0.000000,-23960.050000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000
25%,5.512000,161.228000,2601.264525,1609.440000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.092100,-0.121200,-0.156100
50%,11.680000,531.620000,8483.786200,4959.730000,25.376000,93.251000,71.789000,4.263171e+03,2312.635000,1458.290000,...,0.720000,0.870000,0.000000,0.810000,1.200000,1.850000,1.710000,0.000700,0.000000,0.000000
75%,21.325000,1017.915000,24777.021600,16119.365000,171.956000,519.000000,324.142000,1.832651e+04,11122.379000,7225.340000,...,2.010000,3.140000,3.250000,2.190000,3.290000,5.840000,6.410000,0.137400,0.202200,0.316900
max,5100.124000,15753.833000,785926.700200,627372.870000,58644.360000,113484.000000,96488.000000,1.015142e+06,769421.000000,437061.000000,...,279.220000,391.570000,491.450000,10514.270000,12481.880000,9250.570000,7401.520000,4.099900,5.609100,12.483900


In [24]:
#Get the data we can work with
df_pre_train['date'] = pd.to_datetime(df_pre_train['date'])
df_pre_train = df_raw.loc[df_raw['P/E_ma_5y'] != 0]
df_pre_train = df_pre_train.loc[df_raw['return_3m'] != 0]

In [25]:
#Split the data between prediction y and data X

X = df_pre_train.iloc[:,2:-3]
y_3m = df_pre_train.iloc[:,-3]
y_6m = df_pre_train.iloc[:,-2]
y_1y = df_pre_train.iloc[:,-1]

In [26]:
# Describe the prediction y

y_3m.describe()

count    158987.000000
mean          0.023367
std           0.266076
min          -1.000000
25%          -0.103100
50%           0.016600
75%           0.141500
max           3.670100
Name: return_3m, dtype: float64

## 4. Parameters

In [27]:
# Date intervals

lista_date_start = ['2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01','2013-01-01']
lista_date_end = ['2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01','2018-01-01','2019-01-01','2020-01-01','2021-01-01','2022-01-01','2023-01-01']

# file names
nomes_scalers = ['sc2013.pkl','sc2014.pkl','sc2015.pkl','sc2016.pkl','sc2017.pkl','sc2018.pkl','sc2019.pkl','sc2020.pkl','sc2021.pkl','sc2022.pkl','sc2023.pkl']
nomes_modelos = ['ANN2013','ANN2014','ANN2015','ANN2016','ANN2017','ANN2018','ANN2019','ANN2020','ANN2021','ANN2022','ANN2023']

In [28]:
inferior_short_limit = -0.15

In [40]:
path_to_save_model = os.getcwd()+'\\..\\models\\'
path_to_save_scalers = os.getcwd()+'\\..\\scalers\\'

## 5. Training

In [41]:
for i in range(len(lista_date_start)):
    X_selected = df_pre_train.loc[(df_pre_train['date'] < datetime.strptime(lista_date_end[i], "%Y-%d-%m").date()) & (df_pre_train['date'] > datetime.strptime(lista_date_start[i], "%Y-%d-%m").date())]
    print(lista_date_start[i])
    print(lista_date_end[i])
    X = X_selected.iloc[:,2:-3]
    y = X_selected.iloc[:,-3]
    y = y.apply(lambda x: 1 if x <= inferior_short_limit else 0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X)
    joblib.dump(sc, path_to_save_scalers + nomes_scalers[i])
    
    ANN = tf.keras.models.Sequential()
    ANN.add(tf.keras.layers.Dense(units = 63, activation = 'relu'))
    ANN.add(tf.keras.layers.Dense(units = 63, activation = 'relu'))
    ANN.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
    ANN.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_crossentropy','accuracy'])
    ANN.fit(X_train, y, batch_size = 32, epochs = 30)
    ANN.save(path_to_save_model+nomes_modelos[i])

2003-01-01
2013-01-01
Epoch 1/30
1328/1328 [==============================] - 2s 1ms/step - loss: 0.2797 - binary_crossentropy: 0.2797 - accuracy: 0.8894
Epoch 2/30
1328/1328 [==============================] - 2s 1ms/step - loss: 0.2059 - binary_crossentropy: 0.2059 - accuracy: 0.9176
Epoch 3/30
1328/1328 [==============================] - 2s 2ms/step - loss: 0.1827 - binary_crossentropy: 0.1827 - accuracy: 0.9281
Epoch 4/30
1328/1328 [==============================] - 2s 2ms/step - loss: 0.1709 - binary_crossentropy: 0.1709 - accuracy: 0.9329
Epoch 5/30
1328/1328 [==============================] - 2s 1ms/step - loss: 0.1611 - binary_crossentropy: 0.1611 - accuracy: 0.9374
Epoch 6/30
1328/1328 [==============================] - 3s 2ms/step - loss: 0.1558 - binary_crossentropy: 0.1558 - accuracy: 0.9404
Epoch 7/30
1328/1328 [==============================] - 2s 2ms/step - loss: 0.1511 - binary_crossentropy: 0.1511 - accuracy: 0.9425
Epoch 8/30
1328/1328 [==============================] 

INFO:tensorflow:Assets written to: C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\models\ANN2013\assets


2004-01-01
2014-01-01
Epoch 1/30
1565/1565 [==============================] - 4s 2ms/step - loss: 0.2965 - binary_crossentropy: 0.2965 - accuracy: 0.8800
Epoch 2/30
1565/1565 [==============================] - 4s 3ms/step - loss: 0.2210 - binary_crossentropy: 0.2210 - accuracy: 0.9088
Epoch 3/30
1565/1565 [==============================] - 2s 1ms/step - loss: 0.1995 - binary_crossentropy: 0.1995 - accuracy: 0.9195
Epoch 4/30
1565/1565 [==============================] - 3s 2ms/step - loss: 0.1853 - binary_crossentropy: 0.1853 - accuracy: 0.9261
Epoch 5/30
1565/1565 [==============================] - 4s 2ms/step - loss: 0.1754 - binary_crossentropy: 0.1754 - accuracy: 0.9303
Epoch 6/30
1565/1565 [==============================] - 3s 2ms/step - loss: 0.1709 - binary_crossentropy: 0.1709 - accuracy: 0.9319
Epoch 7/30
1565/1565 [==============================] - 2s 1ms/step - loss: 0.1630 - binary_crossentropy: 0.1630 - accuracy: 0.9348
Epoch 8/30
1565/1565 [==============================] 

INFO:tensorflow:Assets written to: C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\models\ANN2014\assets


2005-01-01
2015-01-01
Epoch 1/30
1786/1786 [==============================] - 3s 1ms/step - loss: 0.2970 - binary_crossentropy: 0.2970 - accuracy: 0.8781
Epoch 2/30
1786/1786 [==============================] - 5s 3ms/step - loss: 0.2274 - binary_crossentropy: 0.2274 - accuracy: 0.9050
Epoch 3/30
1786/1786 [==============================] - 4s 2ms/step - loss: 0.2043 - binary_crossentropy: 0.2043 - accuracy: 0.9148
Epoch 4/30
1786/1786 [==============================] - 4s 2ms/step - loss: 0.1920 - binary_crossentropy: 0.1920 - accuracy: 0.9215
Epoch 5/30
1786/1786 [==============================] - 4s 2ms/step - loss: 0.1833 - binary_crossentropy: 0.1833 - accuracy: 0.9255
Epoch 6/30
1786/1786 [==============================] - 3s 2ms/step - loss: 0.1740 - binary_crossentropy: 0.1740 - accuracy: 0.9295
Epoch 7/30
1786/1786 [==============================] - 5s 3ms/step - loss: 0.1698 - binary_crossentropy: 0.1698 - accuracy: 0.9313
Epoch 8/30
1786/1786 [==============================] 

...

INFO:tensorflow:Assets written to: C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\models\ANN2021\assets


2012-01-01
2022-01-01
Epoch 1/30
3235/3235 [==============================] - 7s 2ms/step - loss: 0.3467 - binary_crossentropy: 0.3467 - accuracy: 0.8483
Epoch 2/30
3235/3235 [==============================] - 8s 2ms/step - loss: 0.2708 - binary_crossentropy: 0.2708 - accuracy: 0.8824
Epoch 3/30
3235/3235 [==============================] - 8s 2ms/step - loss: 0.2462 - binary_crossentropy: 0.2462 - accuracy: 0.8945
Epoch 4/30
3235/3235 [==============================] - 7s 2ms/step - loss: 0.2311 - binary_crossentropy: 0.2311 - accuracy: 0.9014
Epoch 5/30
3235/3235 [==============================] - 5s 2ms/step - loss: 0.2215 - binary_crossentropy: 0.2215 - accuracy: 0.9052
Epoch 6/30
3235/3235 [==============================] - 8s 2ms/step - loss: 0.2115 - binary_crossentropy: 0.2115 - accuracy: 0.9083
Epoch 7/30
3235/3235 [==============================] - 8s 2ms/step - loss: 0.2066 - binary_crossentropy: 0.2066 - accuracy: 0.9108
Epoch 8/30
3235/3235 [==============================] 

INFO:tensorflow:Assets written to: C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\models\ANN2022\assets


2013-01-01
2023-01-01
Epoch 1/30
3370/3370 [==============================] - 10s 2ms/step - loss: 0.3641 - binary_crossentropy: 0.3641 - accuracy: 0.8381
Epoch 2/30
3370/3370 [==============================] - 8s 2ms/step - loss: 0.2891 - binary_crossentropy: 0.2891 - accuracy: 0.8738
Epoch 3/30
3370/3370 [==============================] - 8s 2ms/step - loss: 0.2604 - binary_crossentropy: 0.2604 - accuracy: 0.8889
Epoch 4/30
3370/3370 [==============================] - 6s 2ms/step - loss: 0.2461 - binary_crossentropy: 0.2461 - accuracy: 0.8952
Epoch 5/30
3370/3370 [==============================] - 8s 2ms/step - loss: 0.2341 - binary_crossentropy: 0.2341 - accuracy: 0.8996
Epoch 6/30
3370/3370 [==============================] - 8s 2ms/step - loss: 0.2251 - binary_crossentropy: 0.2251 - accuracy: 0.9028
Epoch 7/30
3370/3370 [==============================] - 6s 2ms/step - loss: 0.2194 - binary_crossentropy: 0.2194 - accuracy: 0.9060
Epoch 8/30
3370/3370 [==============================]

INFO:tensorflow:Assets written to: C:\Users\mathe\Coding Projects\quant-finance-strategy\notebooks\..\models\ANN2023\assets
